In [1]:
from odc.aws.queue import publish_messages, get_queue
from datetime import datetime, timedelta
import os
import json

In [2]:
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"
dead_queue = get_queue("ghrsst-queue-dead")
queue = get_queue("ghrsst-queue")

In [29]:
import tqdm

# For each day of the year, create a timestamp string and publish it to the queue
messages = []
for year in [2024]:
    for day in range(100, 150):
        dt = datetime(year, 1, 1) + timedelta(day - 1)
        body = {
            "date": dt.strftime("%Y-%m-%d"),
        }
        messages.append({"Id": body["date"], "MessageBody": json.dumps(body)})

count = 0
message_group = []
progress = tqdm.tqdm(messages)

for message in progress:
    # Assume this works. Exception handling elsewhere.
    message_group.append(message)
    count += 1

    if count % 10 == 0:
        publish_messages(queue, message_group)
        message_group = []

# Post the last few messages
if len(message_group) > 0:
    publish_messages(queue, message_group)
    progress.update(count)

# Return the number of messages that were re-driven.
print(f"Published {count} messages")

100%|██████████| 50/50 [00:01<00:00, 30.00it/s]

Published 50 messages


In [28]:
dead_queue = get_queue("ghrsst-queue-dead")
queue = get_queue("ghrsst-queue")

print(
    f"Found {dead_queue.attributes.get('ApproximateNumberOfMessages')} messages in dead queue"
)
print(
    f"Found {queue.attributes.get('ApproximateNumberOfMessages')} messages in the alive queue"
)

Found 0 messages in dead queue
Found 27 messages in the alive queue


In [12]:
import os
import boto3
from datetime import datetime, timedelta
import json

N_PREVIOUS_DAYS = 10

QUEUE_NAME = "ghrsst-queue"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"


today = datetime.today()

messages = []

for i in range(N_PREVIOUS_DAYS):
    dt = today - timedelta(days=i)

    body = {
        "date": dt.strftime("%Y-%m-%d"),
    }
    messages.append({"Id": body["date"], "MessageBody": json.dumps(body)})

sqs = boto3.resource("sqs")
queue = sqs.get_queue_by_name(QueueName=QUEUE_NAME)

queue.send_messages(Entries=messages)

{'Successful': [{'Id': '2024-05-27',
   'MessageId': '38073eb0-fb62-46f6-bcdc-c30177220e5e',
   'MD5OfMessageBody': '178957661a429f4d91a8c6ff67a23a38'},
  {'Id': '2024-05-26',
   'MessageId': '6abf9a3e-7604-4854-8b8a-d0d8964e09da',
   'MD5OfMessageBody': '7c8d40188f55527850df60ed42d5927c'},
  {'Id': '2024-05-25',
   'MessageId': 'a4b57f64-a450-4268-a678-78c018c6567f',
   'MD5OfMessageBody': '75e6393bc9b1b8b7bf840289c964640d'},
  {'Id': '2024-05-24',
   'MessageId': 'e3d26e3a-24ab-49fd-a630-7d41c29ba154',
   'MD5OfMessageBody': '5451ff6b6e446aa67f1e5b828aaa3abb'},
  {'Id': '2024-05-23',
   'MessageId': 'a9ef9aea-8100-43ee-8fc9-42e93f014fc6',
   'MD5OfMessageBody': '52d928d73bfdb95c5139b2c9a4305019'},
  {'Id': '2024-05-22',
   'MessageId': '1f6ba19c-aa3f-490f-afe6-de3c977db4c7',
   'MD5OfMessageBody': 'bfb3ce6d78d03226d921a53c1a441e34'},
  {'Id': '2024-05-21',
   'MessageId': '798d8b32-90cd-4110-b124-9c92614dbad5',
   'MD5OfMessageBody': '488e24e2f516e472f5e1161aac26fe7a'},
  {'Id': '202